In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np
import jlc
import scipy
import cv2
from utils import estimate_bias, estimate_bias_coefs,norm_quantile,norm_translation,process_vol,soft_threshold,get_disc_2d,inspect_fiber_vol
from pathlib import Path
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
filenames = list(Path("./data/").glob("*.tif"))
filenames = [str(f) for f in filenames]
print(filenames)

['data\\YCsgA_Exp2_Pos1.tif', 'data\\YCsgA_Exp2_Pos2.tif', 'data\\YCsgA_Exp2_Pos3.tif', 'data\\YCsgA_Exp2_Pos4.tif', 'data\\YCsgA_Exp3_Pos1.tif', 'data\\YCsgA_Exp3_Pos2.tif', 'data\\YCsgA_Exp3_Pos3.tif', 'data\\YCsgA_Exp3_Pos4.tif']


In [ ]:
vol = jlc.load_tifvol(filenames[0])
vol = vol[:,:,:vol.shape[2]//2].astype(float)
vol = norm_quantile(vol,alpha=0.001,clip=True)
coefs = estimate_bias_coefs(vol)
vol *= coefs.reshape(-1,1,1)
bias = estimate_bias(vol)
vol -= bias
vol = norm_quantile(vol,alpha=0.001,clip=True)
vol,best_translation = norm_translation(vol)

In [4]:
images = []
for filename in filenames:
    vol = jlc.load_tifvol(filename.replace("data","data_processed"))
    images.append(vol[vol.shape[0]//2,:,:vol.shape[2]//2].astype(float))
jlc.montage(images)

In [10]:
jlc.montage([np.clip(im,0,1000) for im in images])

In [3]:
#process and save
for filename in filenames:
    savename = filename.replace("data","data_processed2")
    os.makedirs(os.path.dirname(savename),exist_ok=True)
    jlc.save_tifvol(process_vol(filename),savename)
    print("done with ",filename)

done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos1.tif
done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos2.tif
done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos3.tif
done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos4.tif
done

In [4]:
filenames = list(Path("./data_processed/").glob("*.tif"))
filenames = [str(f) for f in filenames]
print(filenames)

['data_processed\\YCsgA_Exp2_Pos1.tif']


In [ ]:
#4,5,6,7: weird in the start
#2: one bad frame in start


In [7]:
vol = jlc.load_tifvol(filenames[0].replace("data","data_processed"))
jlc.inspect_vol(vol.transpose(0,2,1))

In [ ]:
from utils import FiberGraph

fiber_graph = FiberGraph()
n = 10
for i in range(100):
    fiber_graph.add_node(np.random.rand()*n,np.random.rand()*n,i)


plt.imshow(np.zeros((n,n)),cmap="gray",extent=[0,n,0,n])
fiber_graph.plot()

In [28]:
from structure_tensor import eig_special_2d, structure_tensor_2d
image = vol[200]

sigma = 1
rho = 3
S = structure_tensor_2d(image, sigma, rho)
val, vec = eig_special_2d(S)

print(val.shape)
print(vec.shape)
print(S.shape)

(2, 684, 428)
(2, 684, 428)
(3, 684, 428)


In [29]:
plt.figure(figsize=(15, 15))
plt.subplot(221)
plt.imshow(image.transpose(1,0),cmap="gray")
plt.subplot(222)
val3 = np.concatenate((norm_quantile(val[0,:,:,None],alpha=0.01),
                       norm_quantile(val[1,:,:,None],alpha=0.01),
                       0*val[1,:,:,None]),axis=2)
plt.imshow(val3.transpose(1,0,2))
plt.subplot(223)
vec3 = np.concatenate((norm_quantile(vec[0,:,:,None],alpha=0.01),
                       norm_quantile(vec[1,:,:,None],alpha=0.01),
                       0*vec[1,:,:,None]),axis=2)
plt.imshow(vec3.transpose(1,0,2))
plt.subplot(224)
S3 = norm_quantile(S.transpose(2,1,0),alpha=0.01)
plt.imshow(S3)

In [3]:
vol = jlc.load_tifvol(filenames[0].replace("data","data_processed"))
vol = scipy.ndimage.gaussian_filter(vol, sigma=(0, 0.7, 0.7))
jlc.inspect_vol(soft_threshold(vol.transpose(0,2,1),0.1,0.2))

In [8]:
vol = jlc.load_tifvol(filenames[4].replace("data","data_processed")).transpose(0,2,1)
#vol = vol[:,200:300,200:350]
#vol = scipy.ndimage.gaussian_filter(vol, sigma=(0, 1, 1))
#nereast
vol = scipy.ndimage.median_filter(vol, size=(9,1,1), mode='nearest')


In [34]:
sigma1 = 0.75
sigma2 = 1.5
vol = scipy.ndimage.gaussian_filter(vol, sigma=(0, sigma1, sigma1))-scipy.ndimage.gaussian_filter(vol, sigma=(0, sigma2, sigma2))
vol[vol<0] = 0
vol = norm_quantile(vol,alpha=0.001,clip=True)
jlc.inspect_vol(vol)
jlc.zoom()

In [37]:
jlc.inspect_vol(vol,vmin=0,vmax=0.1)
jlc.zoom()

In [4]:
# TODO
# Favor newer fibers when figuring out fiber origin
# filter out fibers that moved/dissapeard
# add calculation for fiber direction and include it in fiber origin selection
# dont let fibers do tiny branches
# if a node is very close to an edge and part of the CC, simply adjust the edge

In [20]:

def find_conn2(xy,nodes,node,r,upper_limit=15,at_most_prev_nodes=10,at_most_prev_t=100,good_speed=0.15):
    dist = np.linalg.norm(xy-np.array([node["x"],node["y"]]),axis=1)
    dist_idx = np.where(dist<=upper_limit)[0]
    dist_idx = np.array([i for i in dist_idx if nodes[i]["active"]])
    if len(dist_idx)==0:
        return []
    #split into growing and non-growing:
    growing_index = np.array([i for i in dist_idx if nodes[i]["growing"]])
    cost = []
    t_now = node["t"]
    for i in growing_index:
        path = [nodes[i]]
        path_idx = [i]
        for _ in range(at_most_prev_nodes):
            if len(path[-1]["conn"])==0:
                break
            j = path[-1]["conn"][0]
            if nodes[j]["root"]:
                break
            path.append(nodes[j])
            path_idx.append(j)
            if path[-1]["t"]<t_now-at_most_prev_t:
                break
        if not len(path)==len(np.unique(path_idx)):
            print(path_idx)
            for j in np.unique(path_idx):
                print("NODE "+str(j))
                print(nodes[j])
            raise Exception("path_idx not unique")
        if len(path)>1:
            xy_path = np.array([[n["x"],n["y"]] for n in path])
            cum_dist = np.linalg.norm(xy_path[1:]-xy_path[:-1],axis=1).sum()
            cum_t = t_now-path[-1]["t"]
            speed = cum_dist/cum_t
            gs = good_speed
            speed = 1-soft_threshold(speed,0.5*gs,1.5*gs)
            speed *= 1-soft_threshold(len(path),0,3)
            v1 = np.array([node["x"],node["y"]])-xy_path[0]
            v2 = xy_path[1]-xy_path[0]
            angle = np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))
            angle = 1-angle/np.pi
            angle = 1-soft_threshold(angle,0.6,0.9,2)
        else:
            angle = 0
            speed = 0
        #cost formula
        cost.append(dist[i]*(1+1.0*speed)+1.0*angle)
    if node["x"]==104 and node["y"]==36:
        print(cost)
    if any([c<r*3+4 for c in cost]):
        return [growing_index[np.argmin(cost)]]
    else:
        non_growing_index = np.array([i for i in dist_idx if not nodes[i]["growing"]])
        if len(non_growing_index)==0:
            return []
        cost = dist[non_growing_index]
        if any(cost<r*3+2):
            return [non_growing_index[np.argmin(cost)]]
        else:
            return []


In [9]:
def find_conn3(xy,nodes,node,r,growing_reward=2):
    dist = np.linalg.norm(xy-np.array([node["x"],node["y"]]),axis=1)
    t = np.array([n["t"] for n in nodes if n["active"]])
    t_now = node["t"]
    mod_dist = dist#-5.0*self.radius*(1-soft_threshold(t_now-t-1,0,10,3))
    active_idx = np.array([n["idx"] for n in nodes if n["active"]])
    mod_dist = mod_dist[active_idx]
    growing_idx = np.array([k for (k,i) in enumerate(active_idx) if nodes[i]["growing"]])
    mod_dist[growing_idx] -= growing_reward
    if any(mod_dist<r*3+2):
        return [active_idx[np.argmin(mod_dist)]]
    else:
        return []

In [96]:
from utils import FiberGraph
from functools import partial
fiber_graph = FiberGraph(radius=2,find_conn_func=find_conn3,threshold=0.2)
mf = vol.shape[0]
fiber_graph.process_vol(vol,max_frames=mf)
plt.figure()
plt.imshow(vol[mf-1],cmap="gray")
fiber_graph.plot()
jlc.zoom()

In [98]:
inspect_fiber_vol(vol*0.3,fiber_graph=fiber_graph)
jlc.zoom()

In [93]:
soft_max = soft_threshold(scipy.ndimage.gaussian_filter(vol[-1],sigma=0.7),0.1,0.2)
cmap_image = (soft_max*(vol>0.15).mean(0)*255).astype(np.uint8)
cmap = plt.get_cmap("jet")
cmap = np.array([cmap(i)[:3] for i in np.linspace(0,1,256)])
cmap[0,:] = 0
cmap_image = cmap[cmap_image]
plt.imshow(cmap_image)
jlc.zoom()

In [90]:
vol = jlc.load_tifvol(filenames[0].replace("data","data_processed")).transpose(0,2,1)
#vol = vol[:,200:300,200:350]
#vol = scipy.ndimage.gaussian_filter(vol, sigma=(0, 1, 1))
#nereast
vol = scipy.ndimage.median_filter(vol, size=(9,1,1), mode='mirror')

In [91]:
def laplace(x,s1=0.75,s2=1.5,clip0=True):
    x = scipy.ndimage.gaussian_filter(x, sigma=(0, s1, s1))-scipy.ndimage.gaussian_filter(x, sigma=(0, s2, s2))
    if clip0:
        x[x<0] = 0
    return x

vol = np.maximum(norm_quantile(laplace(vol),0.01)*soft_threshold(vol,0.08,0.12),soft_threshold(vol,0.5,0.75))
jlc.inspect_vol(vol)
jlc.zoom()